In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import tensorflow as tf
from livelossplot import PlotLossesKeras
np.random.seed(7)
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LSTM
from keras.layers.embeddings import Embedding
from keras.utils import np_utils
from keras.preprocessing import sequence
from gensim.models import Word2Vec, KeyedVectors, word2vec
import gensim
from gensim.utils import simple_preprocess
from tensorflow.keras.utils import to_categorical
import pickle
import h5py
from time import time

In [ ]:
#filename = 'C:/Users/Bartosz/OneDrive/magisterka semestr 3/UM/projekt nlp/ml-agh-course/data/polish_sentiment_dataset.csv'
filename = 'C:/Users/Bartosz/OneDrive/magisterka semestr 3/UM/dane_nowe.csv'
dataset = pd.read_csv(filename, delimiter = ";")

# Delete unused column
#del dataset['length']

# Delete All NaN values from columns=['description','rate']
dataset = dataset[dataset['description'].notnull() & dataset['rate'].notnull()]

# We set all strings as lower case letters
#dataset['description'] = dataset['description'].str.lower()



In [ ]:
#class_names = ["Neutral","Positive", "Negative"]

In [ ]:
dataset.head()

In [ ]:
print(dataset.count())

temp = dataset.groupby('rate').count()['description'].reset_index().sort_values(by='description',ascending=False)

print(temp)

In [ ]:
#making dataset more symmetric

dataset_negative = dataset[dataset['rate'] == -1.0]
print(dataset_negative.count())
dataset_positive = dataset[dataset['rate'] == 1.0]
print(dataset_positive.count())
dataset_neutral = dataset[dataset['rate'] == 0.0]
print(dataset_neutral.count())

dataset_positive_cut=dataset_positive[:183391]
print(dataset_positive_cut.count())



In [ ]:
##making dataset more symmetric

print(dataset_positive_cut.count())
frames = [dataset_positive_cut, dataset_negative, dataset_neutral]

dataset = pd.concat(frames)
print(dataset.count())

In [ ]:
stop_words_path = 'C:/Users/Bartosz/OneDrive/magisterka semestr 3/UM/polishstopwords.txt'


stop_words = pd.read_csv(stop_words_path, sep=" ", header=None)
stop_words.columns = ["word"]

stop_words.head()

stop_words = stop_words["word"].tolist()

print(stop_words)

In [ ]:
import re
import string

def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    #lowecasre
    text = str(text).lower()
    #remove text in square brackets
    text = re.sub('\[.*?\]', '', text)
    #remove links
    text = re.sub('https?://\S+|www\.\S+', '', text)
    #remove punctuation and words containing numbers
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    #removing stop words
    tokens = []
    for token in text.split():
        if token not in stop_words:
                tokens.append(token)
    return " ".join(tokens)
    

test = "hej, nie co tam i albo aczkolwiek, polska gola lewandowski"

print(clean_text(test))
    


In [ ]:
#text preprocessing

dataset['description'] = dataset['description'].apply(lambda x:clean_text(x))

dataset.head()

In [ ]:
from wordcloud import WordCloud
#from many_stop_words import get_stop_words

#stop_words = get_stop_words('pl')

wordcloud = WordCloud(
                    background_color='white',
                    stopwords=stop_words,
                    max_words=100,
                    max_font_size=40,
                    random_state=42
).generate(str(dataset['description']))

print(wordcloud)
fig = plt.figure(1)
plt.imshow(wordcloud)
plt.axis('off')
plt.savefig('Plots/woirdcloud.png')
plt.show()

In [ ]:
from collections import Counter

dataset['description'] = dataset['description'].apply(lambda x:str(x).split())
top = Counter([item for sublist in dataset['description'] for item in sublist])
temp = pd.DataFrame(top.most_common(20))
temp = temp.iloc[1:,:]
temp.columns = ['Common_words','count']
temp.style.background_gradient(cmap='Purples')

In [ ]:
X = dataset['description']
y = dataset['rate']

#splitting dataset to train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#splitting train to train and validate
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
print(y_train)
print(y_val)

dlugosc poszczegolnych zbiorow


In [ ]:
print("X_train shape: " + str(X_train.shape))
print("X_test shape: " + str(X_test.shape))
print("X_val shape: " + str(X_val.shape))
print("y_train shape: " + str(y_train.shape))
print("y_test shape: " + str(y_test.shape))
print("y_val shape: " + str(y_val.shape))


stworzenie wektoru słów na bazie modelu z polskiej akademii nauk


In [ ]:
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('nkjp.txt', binary=False)



Zamiana syn0 na vectors


In [ ]:
emb_mean, emb_std = word2vec_model.vectors.mean(), word2vec_model.vectors.std()



In [ ]:
embedding_matrix = word2vec_model.vectors
print('Shape of embedding matrix: ', embedding_matrix.shape)

test

In [ ]:
word2vec_model.most_similar("siema")

Vectorize X_train and X_test to 2D tensor:

fit_on_texts Updates internal vocabulary based on a list of texts. This method creates the vocabulary index based on word frequency. So if you give it something like, "The cat sat on the mat." It will create a dictionary s.t. word_index["the"] = 1; word_index["cat"] = 2 it is word -> index dictionary so every word gets a unique integer value.

In [ ]:
top_words = embedding_matrix.shape[0]
mxlen = 50
nb_classes = 3

tokenizer = Tokenizer(num_words=top_words)
tokenizer.fit_on_texts(X_train)


texts_to_sequences Transforms each text in texts to a sequence of integers. So it basically takes each word in the text and replaces it with its corresponding integer value from the word_index dictionary

In [ ]:
sequences_train = tokenizer.texts_to_sequences(X_train)
sequences_test = tokenizer.texts_to_sequences(X_test)
sequences_val = tokenizer.texts_to_sequences(X_val)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
print(word_index)



In [ ]:
print(X_train)

Pads sequences to the same length. Converts a class vector (integers) to binary class matrix.

In [ ]:
X_train = sequence.pad_sequences(sequences_train, maxlen=mxlen)
X_test = sequence.pad_sequences(sequences_test, maxlen=mxlen)
X_val = sequence.pad_sequences(sequences_val, maxlen=mxlen)

#binary encoding
y_train = np_utils.to_categorical(y_train, nb_classes)
y_test = np_utils.to_categorical(y_test, nb_classes)
y_val = np_utils.to_categorical(y_val, nb_classes)

X_train

In [ ]:
print(y_train.shape)


print(y_train)

In [ ]:
class_names = ["Neutral","Positive", "Negative"]

In [ ]:
print(X_train)

creating  LST network model (kind of RNN)

In [ ]:
batch_size = 32
nb_epoch = 15

embedding_layer = Embedding(embedding_matrix.shape[0],
                            embedding_matrix.shape[1],
                            weights=[embedding_matrix],
                            trainable=False)

model = Sequential()
model.add(embedding_layer)
model.add(LSTM(128, recurrent_dropout=0.3, dropout=0.3))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))
model.summary()

We just need to compile the model and we will be ready to train it. When we compile the model, we declare the optimizer (Adam, SGD, etc.)  and the loss function. To fit the model, all we have to do is declare the number of epochs and the batch size.

nb_epochs renamed to epochs

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
rnn = model.fit(X_train, y_train, epochs= 15, batch_size=batch_size, shuffle=True, validation_data=(X_val, y_val))
score = model.evaluate(X_val, y_val)
print("Test Loss: %.2f%%" % (score[0]*100))
print("Test Accuracy: %.2f%%" % (score[1]*100))

saving model


In [ ]:
print('Save model...')
model.save('Models/finalsentimentmodelv3.h5')
print('Saved model to disk...')

print('Save Word index...')
output = open('Models/finalwordindexv3.pkl', 'wb')
pickle.dump(word_index, output)
print('Saved word index to disk...')

In [ ]:
print(rnn.history)
pd.DataFrame(rnn.history).plot(figsize=(10, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

krzywe uczenia


In [ ]:
plt.figure(0)
plt.plot(rnn.history['accuracy'],'r')
plt.plot(rnn.history['val_accuracy'],'g')
plt.plot(rnn.history['loss'],'y')
plt.plot(rnn.history['val_loss'],'b')
plt.xticks(np.arange(0, nb_epoch+1, nb_epoch/5))
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Num of Epochs")
plt.ylabel("Accuracy")
plt.title("Training vs Validation Accuracy LSTM , epochs=15") # for max length = 10 and 20 epochs
plt.legend(['train_accuracy', 'val_accuracy','train_loss', 'val_loss'], loc=5, prop={'size': 14})

# plt.figure(1)
# plt.plot(rnn.history['loss'],'r')
# plt.plot(rnn.history['val_loss'],'g')
# plt.xticks(np.arange(0, nb_epoch+1, nb_epoch/5))
# plt.rcParams['figure.figsize'] = (8, 6)
# plt.xlabel("Num of Epochs")
# plt.ylabel("Training vs Validation Loss LSTM, epochs=15") # for max length = 10 and 20 epochs
# plt.legend(['train', 'validation'])
plt.savefig('Plots/accuracy_loss.png')
plt.show()

In [ ]:
# Apply Precision-Recall

from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

y_pred = model.predict(X_test)

# Convert Y_Test into 1D array
yy_true = [np.argmax(i) for i in y_test]
print(yy_true)

yy_scores = [np.argmax(i) for i in y_pred]
print(yy_scores)

print("Recall: " + str(round(recall_score(yy_true, yy_scores, average='weighted'),2)))
print("Precision: " + str(round(precision_score(yy_true, yy_scores, average='weighted'),2)))
print("F1 Score: " + str(round(f1_score(yy_true, yy_scores, average='weighted'),2)))

In [ ]:
# Apply Confusion matrix

from sklearn.metrics import classification_report, confusion_matrix
y_pred = model.predict(X_test, verbose=2)
y_pred = np.argmax(y_pred, axis=1)

for ix in range(3):
    print(ix, confusion_matrix(np.argmax(y_test, axis=1), y_pred)[ix].sum())
cm = confusion_matrix(np.argmax(y_test, axis=1), y_pred)
print(cm)

# Visualizing of confusion matrix
import seaborn as sn

df_cm = pd.DataFrame(cm, range(3), range(3))
plt.figure(figsize=(10,7))
sn.set(font_scale=1.4)
sn.heatmap(df_cm, annot=True,fmt='d')
sn.set_context("poster")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.savefig('Plots/confusionMatrix.png')
plt.show()

In [ ]:
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import pickle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras import models


model = load_model('Models/finalsentimentmodelv3.h5')
model.summary()

with open('Models/finalwordindexv3.pkl', 'rb') as picklefile:
    word_index = pickle.load(picklefile)
top_words = len(word_index)
tokenizer = Tokenizer(num_words=top_words)
tokenizer.word_index = word_index
print(word_index)


print('Found %s uniqe tokens.' % len(word_index))





In [ ]:
# Insert text for example 'your sentence in Polish'
text = ['ogólnie to jest bardzo beznadziejne pozdrawiam']


text[0] = clean_text(text[0])
print(text)

test_sequences = tokenizer.texts_to_sequences(text)

x_test = sequence.pad_sequences(test_sequences, maxlen=40)

print('x_test shape:', str(x_test.shape))
#model.load_weights('Models/finalsentimentmodel.h5')
result = model.predict(x_test)
print(result)
print("Neutral: %.2f%%" % (result[:,0]*100))
print("Positive: %.2f%%" % (result[:,1]*100))
print("Negative: %.2f%%" % (result[:,2]*100))
#print(result)


class_names[np.argmax(result)]
